# IFN647 Assignment 1
- Student Number: n10515402
- First Name: Aidan
- Last Name: Lockwood


Loading in the required packages

In [300]:
import string

import numpy as np
from stemming.porter2 import stem
import os 

# Question 1. Parsing of Documents & Queries 
The motivation for Question 1 is to design your own document and query parsers. So please don't use python packages that we didn't use in the workshop. 

## Task 1.1:
Define a document parsing function `parse_docs(stop_words, inputfolder)` to parse a data collection (e.g. RCV1v3 dataset), where parameter `stop_words` is a list of common English words (you may use the file `common-english-words.txt` to find all stop words), and parameter `inputfolder` is the folder that stores the set of XML files.

The following are the major steps in the document parsing function:
### Step 1 
The function reads XML files from inputfolder (e.g., RCV1v3). For each XML file, it finds the document ID and index terms and then represents it in a DocV3 Object. 
You need to define a DocV3 class by using Bag-of-Words to represent a document:
- DocV3 needs a document ID variable (attribute newsID), which is simply assigned by the value of ‘itemid’ in <newsitem …> of the XML file.
- In this task, `DocV3` can be initialized with three attributes: `newsID` attribute, an empty dictionary (the attribute name is terms) of key-value pair of (String term: int frequency); and doc_size (the document length) attribute.
- You may define your own methods, e.g., `getNewsID()` to get the document ID, get_termList() to get a sorted list of all terms occurring in the document, etc.

### Step 2 
It then builds up a collection of `DocV3` objects for the given dataset, this collection can be a dictionary structure (as we used in the workshop), a linked list, or a class Rcv1Coll for storing a collection of `DocV3` objects. Please note the rest descriptions are based on the dictionary structure with newsID as key and DocV3 object as value.

### Step 3 
At last, it returns the collection of DocV3 objects.
You also need to follow the following specification to define this parsing function:
- Please use the basic text pre-processing steps, such as tokenizing, stopping words
removal and stemming of words.
Tokenizing –
- You are required to provide definitions of words and terms and describe them as
comments in your Python solution.
- You need to tokenize at least the ‘<text>…</text>’ part of document, exclude all
tags, and discard punctuations and/or numbers based on your definition of terms.
- Define method add_term() for class DocV3 to add new term or increase term
frequency when the term occur again.
Stopping words removal and stemming of terms –
- Use the given stopping words list (“common-english-words.txt”) to ignore/remove
all stopping words. Open and read the given file of stop-words and store them into a
list stopwordList. When adding a term, please check whether the term exists in the
stopwordList, and ignore it if it is in the stopwordList. Note that you can update the
"common-english-words.txt" file based on your data exploration of the dataset, but
you will need to comment the change in your solution.
- You can use porter2 stemming algorithm or other algorithms to update DocV3’s
terms.

Utility Functions Used within the parsing function

In [301]:
def get_docid(file_):

    for line in file_:
        line = line.strip()

        if line.startswith("<text>"):
            start_end = True
        
        if line.startswith("<newsitem "):
            for part in line.split():
                if part.startswith('itemid='):
                    docid = part.split('=')[1].split('/')[0]
                    docid = docid.replace('"', '')

    return docid

def get_stop_words(stop_words):
    
    stop_words_file = open(stop_words, 'r')
    stop_words_list = stop_words_file.readlines()
    
    stop_words_file.close()

    stop_words_list = stop_words_list[0].split(',')
    return stop_words_list

def get_file_contents(file_name):  
    
    opened_file = open(file_name, 'r')
    contents = opened_file.readlines()
    opened_file.close()

    return contents

def parse_text(file_contents):

    start_end = False
    parsed_text = []

    for line in file_contents:
        line = line.strip()

        if line.startswith("<text>"):
            start_end = True

        if line.startswith('<p>'):
            line = line.replace('<p>', '').replace('</p>', '')
            line = line.translate(str.maketrans('', '', string.punctuation))
            line = line.replace('quot', '')

        elif line.startswith('</text>'):
            start_end = False
        
        if start_end:
            parsed_text.append(line)
        

    return parsed_text

DocV3 Class (with a linked list class to store them)

In [302]:
class DocV3:
    def __init__(self, itemid, next = None):
        self.itemid = itemid
        self.terms = {}
        self.doc_size = 0

        self.next = next
    
    def get_news_id(self, itemid):
        self.itemid = itemid
        return self.itemid
    
    def get_doc_size(self, word_count):
        self.doc_size = word_count
        return self.doc_size
    
    def get_terms_list(self):        
        return self.terms
    

    def add_terms_list(self, terms_list):
        self.terms = terms_list  
        return self.terms
    
    
class List_Docs:
    def __init__(self, hnode = None):
        self.head = hnode

    def insert(self, nnode):
        if not self.head:
            self.head = nnode
            return
        current = self.head
        while current.next:
            current = current.next
        current.next = nnode

        return nnode

    # Need to add a printing function here    
    def print_list(self):
        current = self.head
        while current:
            print(f'Document {current.itemid} contains {len(current.terms)} terms, with a total word count of {current.doc_size} words')

            for term in current.terms:
                print(f'{term} : {current.terms[term]}')
            current = current.next

    def get_list(self):
        current = self.head
        doc_list = []
        while current:
            doc_list.append(current)
            current = current.next

        return doc_list
    
    def get_list_size(self):
        current = self.head

        size = 0

        while current:
            current = current.next
            size += 1

        return size

Main Parsing Function

In [303]:
def parse_docs(stop_words, inputfolder):
    stop_words_list = get_stop_words(stop_words)

    input_folder_files = os.listdir(inputfolder)

    # list of DocV3 objects - this will turn into a linked list
    docs_list = List_Docs()

    for path in input_folder_files:
        opened_file = get_file_contents(f'{inputfolder}{path}')

        current_doc = {}

        word_count = 0

        for line in opened_file:
            
            # Retrieve the docid and initialise a new instance of DocV3
            docid = get_docid(opened_file)
            doc_obj = DocV3(docid)

        parsed_text = parse_text(opened_file)

        split_text = []

        for line in parsed_text:
            for word in line.split():
                word_count += 1

                if word.lower() not in stop_words_list and not word.isdigit():
                    word = word.lower()
                    split_text.append(stem(word))
        
        split_text.pop(0)      

        for word in split_text:
            if word not in current_doc:
                current_doc[word] = 1
            else:
                current_doc[word] += 1

        # Order the current_doc dictionary by descending frequency
        current_doc = dict(sorted(current_doc.items(), key=lambda item: item[1], reverse=True))
        
        doc_obj.add_terms_list(current_doc)
        doc_obj.get_doc_size(word_count)

        docs_list.insert(doc_obj)        

    return docs_list

In [304]:
document_folder = 'RCV1v3/'

docs_v3_list = parse_docs('common-english-words.txt', document_folder)

docs_v3_list.print_list()

Document 741299 contains 93 terms, with a total word count of 191 words
open : 3
up : 3
lead : 3
car : 3
lehto : 2
soper : 2
victori : 2
german : 2
schneider : 2
second : 2
over : 2
struggl : 2
tyre : 2
belgian : 2
jj : 1
finland : 1
steve : 1
britain : 1
drove : 1
ail : 1
mclaren : 1
fifth : 1
round : 1
world : 1
gt : 1
championship : 1
sunday : 1
beat : 1
merced : 1
bernd : 1
austrian : 1
alexand : 1
wurz : 1
enabl : 1
16point : 1
overal : 1
stand : 1
mount : 1
strong : 1
challeng : 1
leader : 1
final : 1
minut : 1
fourhour : 1
race : 1
handl : 1
caus : 1
broken : 1
undertray : 1
manag : 1
hold : 1
win : 1
dure : 1
midrac : 1
downpour : 1
ardenn : 1
mountain : 1
thought : 1
everyon : 1
drive : 1
dryweath : 1
joke : 1
afterward : 1
swap : 1
rain : 1
exact : 1
right : 1
time : 1
push : 1
hard : 1
big : 1
third : 1
finish : 1
porsch : 1
franc : 1
bob : 1
wollek : 1
yannick : 1
dalma : 1
thierri : 1
boutsen : 1
former : 1
formula : 1
one : 1
driver : 1
switch : 1
normal : 1
share : 1
han

## Task 1.2 
Define a query parsing function `Parse_Q(query, stop_words)`, where we assume the original query is a simple sentence or a title in a String format (<i>query</i>), and stop_words is a list of stop words that you can get from 'common-english-words.txt.

For example, let query = 

'US EPA ranks Geo Metro car most fuel-efficient 1998 car.'

This function will return a dictionary:
{ 'epa' : 1, 'rank' : 1, 'geo' : 1, 'metro' : 1, 'car' : 2, 'fuel' : 1, 'effici' : 1}

Please note that you should use the same text transformation technique as the document, i.e., tokenising steps for queries **must be identical** to steps for documents and use three different queries to test the function.

In [305]:
def parse_q(query, stop_words):

    query_dict = {}

    stop_words_list = get_stop_words(stop_words)
    
    query = query.strip()

    punctuation = string.punctuation.replace('-', '')

    query = query.translate(str.maketrans('', '', punctuation))

    for word in query.split():

        hiphen_split = word.split('-')

        for compound_word in hiphen_split:
            if compound_word.lower() not in stop_words_list and not compound_word.isdigit():
                compound_word = stem(compound_word.lower())

                if compound_word not in query_dict:
                    query_dict[compound_word] = 1
                else:
                    query_dict[compound_word] += 1
        
    return query_dict

In [306]:
query = parse_q('The quick brown fox jumped over the red-bull car...', 'common-english-words.txt')
print('Query:', query)

Query: {'quick': 1, 'brown': 1, 'fox': 1, 'jump': 1, 'over': 1, 'red': 1, 'bull': 1, 'car': 1}


## Task 1.3
Define a main function to test function `Parse_Docs()` and `Parse_Q()`. The main function uses the provided dataset, calls function `Parse_Docs()` to get a collection of `DocV3` objects. For each document in the collection, firstly print out its `newsID`, the number of terms and the total number of words in the document (<i>doc_size</i>). It then sorts the terms (by frequency) and prints out a <i>term:freq</i> list. At last, it saves the output into a text file (file name is: "your full name_Q1.txt).

In [307]:
def test_parsing(query, stop_words):
    docs_v3_list = parse_docs(stop_words, 'RCV1v3/')

    docs_v3_list.print_list()
    print('-----------------')
    print('Query: ', query)

    print('The parsed query: ')
    parsed_query = parse_q(query, stop_words)
    print(parsed_query)
     

In [308]:
query = 'US EPA ranks Geo Metro car most fuel-efficient 1997 car.'

test_parsing(query, 'common-english-words.txt')

Document 741299 contains 93 terms, with a total word count of 191 words
open : 3
up : 3
lead : 3
car : 3
lehto : 2
soper : 2
victori : 2
german : 2
schneider : 2
second : 2
over : 2
struggl : 2
tyre : 2
belgian : 2
jj : 1
finland : 1
steve : 1
britain : 1
drove : 1
ail : 1
mclaren : 1
fifth : 1
round : 1
world : 1
gt : 1
championship : 1
sunday : 1
beat : 1
merced : 1
bernd : 1
austrian : 1
alexand : 1
wurz : 1
enabl : 1
16point : 1
overal : 1
stand : 1
mount : 1
strong : 1
challeng : 1
leader : 1
final : 1
minut : 1
fourhour : 1
race : 1
handl : 1
caus : 1
broken : 1
undertray : 1
manag : 1
hold : 1
win : 1
dure : 1
midrac : 1
downpour : 1
ardenn : 1
mountain : 1
thought : 1
everyon : 1
drive : 1
dryweath : 1
joke : 1
afterward : 1
swap : 1
rain : 1
exact : 1
right : 1
time : 1
push : 1
hard : 1
big : 1
third : 1
finish : 1
porsch : 1
franc : 1
bob : 1
wollek : 1
yannick : 1
dalma : 1
thierri : 1
boutsen : 1
former : 1
formula : 1
one : 1
driver : 1
switch : 1
normal : 1
share : 1
han

## Question 2: TF*IDF-based IR Model
TF*IDF is a popular term weighting method, which uses the following Eq. (1) to calculate a weight for term k in a document i, where the base of log is e. You may review lecture notes to get the meaning of each variable in the equation.

<img src="./Screenshot 2025-04-09 at 9.36.48 am.png" alt="d_ik equation">

### Task 2.1
Define a function `df(coll)`to calculate document-frequency for a given `DocV3` collection `coll` and return a {term:df, ...} dictionary. You need to test this function and produce a similar output as shown below.

In [309]:
def df(coll):
    term_freq = {}

    for doc in coll.get_list():
        for term in doc.terms:
            try:
                term_freq[term] += 1
            except KeyError:
                term_freq[term] = 1

    print(f'There are {coll.get_list_size()} documents in this data set and contains {len(term_freq)} unique terms.')
    
    term_freq = dict(sorted(term_freq.items(), key=lambda item: item[1], reverse=True))
    
    return term_freq

In [310]:
df(docs_v3_list)

There are 15 documents in this data set and contains 1140 unique terms.


{'corp': 9,
 'one': 8,
 'car': 7,
 'over': 7,
 'market': 7,
 'percent': 7,
 'group': 7,
 'compani': 7,
 'inc': 7,
 'drive': 6,
 'sale': 6,
 'year': 6,
 'sport': 6,
 'unit': 6,
 'design': 6,
 'report': 6,
 'buy': 6,
 'posit': 6,
 'up': 5,
 'manag': 5,
 'share': 5,
 'follow': 5,
 'chang': 5,
 'top': 5,
 'activ': 5,
 'reuter': 5,
 'model': 5,
 'vehicl': 5,
 'motor': 5,
 'sportutil': 5,
 'toyota': 5,
 'two': 5,
 'suzuki': 5,
 'co': 5,
 'rav4': 5,
 'manual': 5,
 'transmiss': 5,
 'consum': 5,
 'statement': 5,
 'honda': 5,
 'three': 5,
 'fourwheel': 5,
 'advertis': 5,
 'product': 5,
 'new': 5,
 'power': 5,
 'aim': 5,
 'part': 5,
 'compet': 5,
 'open': 4,
 'steve': 4,
 'handl': 4,
 'time': 4,
 'push': 4,
 'bob': 4,
 'toronto': 4,
 'bank': 4,
 'london': 4,
 'first': 4,
 'royal': 4,
 'deal': 4,
 'bureau': 4,
 'state': 4,
 'now': 4,
 'mark': 4,
 'repres': 4,
 'protect': 4,
 'presid': 4,
 'news': 4,
 'peopl': 4,
 'countri': 4,
 'further': 4,
 'air': 4,
 'space': 4,
 'spokeswoman': 4,
 'creat': 4,


## Task 2.2
Use Eq. (1) to define a function `tfidf(doc, d_f, ndocs)` to calculate TF*IDF value (weight) of every term in a DocV3 object or a dictionary of {term:freq, ...} d_f is a {term:df, ...} dictionary and ndocs is the number of documents in a given `DocV3` collection. The function returns a {term: tfidf_weight, ...} dictionary for the given document doc.

In [ ]:
def tfidf(doc, d_f, ndocs):
    # The normalised tfidf scores
    d_ik_scores = {}

    # To hold the terms with the docids in which they appear
    coll_frequency_dict = {}

    # For the collection: Indexing the lists, finding each term and counting the frequency of documents each term appears in
    for document in d_f.get_list():
        docid = document.itemid

        for term in document.terms:
            if term not in coll_frequency_dict:
                coll_frequency_dict[term] = {docid : document.terms[term]}
            else:
                coll_frequency_dict[term][docid] = document.terms[term]
    
    n_k_values = {}

    # This is the n_k values for each term in the collection
    for term in coll_frequency_dict:
        n_k_values[term] = len(coll_frequency_dict[term])
    
    f_ik_values = {}

    # For the document: Indexing the document and getting the frequency
    for term in doc.get_terms_list():
        if term not in f_ik_values:
            f_ik_values[term] = doc.terms[term]
        else:
            f_ik_values[term] += doc.terms[term]

    # Computing d_ik for each term in the dictionary
    for term in doc.get_terms_list():
        if term in coll_frequency_dict:
            d_ik = (np.log(f_ik_values[term]) + 1) * np.log(ndocs / n_k_values[term])
            # Normalise the d_ik score
            d_ik = d_ik / np.sqrt(sum([((np.log(f_ik_values[term_t]) + 1) * np.log(ndocs / n_k_values[term_t])) ** 2 for term_t in doc.get_terms_list()]))

            d_ik_scores[term] = d_ik
        else:
            d_ik_scores[term] = 0


    return d_ik_scores

In [320]:
d_f = parse_docs('common-english-words.txt', document_folder)
test_doc = d_f.get_list()

for term in test_doc:
    print(term)
# tfidf(test_doc, d_f, d_f.get_list_size())

## Task 2.3
Define a main function to call <i>tfidf()</i> and **print out top 20 terms** (with its value of tf*idf weight) for each document in <i>RCV1v3</i> if it has more than 20 terms and save the output into a text file (file name is "your full name_Q2.txt")

- You also need to implement a TF*IDF based IR model
- You can assume titles of XML document (the <title>...</title> part) are the original queries, and test at least three titles
- You need to use function `Parse_Q()` that you defined for Question 1 to parse original queries 
- For each query <i>Q</i>, please use the abstract model of ranking (Eq. (2)) to calculate a ranking score for each document <i>D</i>.

Eq. 2.

<img src="Screenshot 2025-04-10 at 3.45.18 pm.png"/>

This task does not require you to use cosine similarity to rank documents for a given query <i>Q</i>, but you do need to provide your observations to see whether the ranking results using cosine are similar to the ranking results using the abstract ranking model. **Hint:** you may calculate the magnitude for each document vector <i>D</i>.

At last, append the output (in descending order) into the text file ("your full name_Q2.txt")


In [ ]:
def top_20_results(d_f):
    # Call the tfidf function to get the scores
    documents = d_f.get_list()

    for doc in documents:
        if len(doc.terms) > 20:
            tfidf_score = tfidf(doc, d_f, d_f.get_list_size())
            # Ensure the scores are in descending order
            tfidf_score = dict(sorted(tfidf_score.items(), key=lambda item: item[1], reverse=True))
            # Get the top 20 terms
            top_20_terms = list(tfidf_score.items())[:20]

            print(f'The top 20 terms for document {doc.itemid} are:')
            for term, score in top_20_terms:
                print(f'{term} : {score}')
            
            print('----------------------------------')
            print('----------------------------------')

    # Implement either a boolean or vector space model

    # Use Parse_Q to parse the queries

    # For each query Q, use the abstract model of ranking to calculate a ranking score for each document D
    # return tfidf_scores

In [338]:
print(top_20_results(d_f))


The top 20 terms for document 741299 are:
lehto : 0.17590165759199103
soper : 0.17590165759199103
victori : 0.17590165759199103
german : 0.17590165759199103
schneider : 0.17590165759199103
second : 0.17590165759199103
struggl : 0.17590165759199103
tyre : 0.17590165759199103
belgian : 0.17590165759199103
lead : 0.1622201762211958
open : 0.1064147818020649
jj : 0.10389035259995417
finland : 0.10389035259995417
drove : 0.10389035259995417
ail : 0.10389035259995417
mclaren : 0.10389035259995417
fifth : 0.10389035259995417
gt : 0.10389035259995417
championship : 0.10389035259995417
beat : 0.10389035259995417

The top 20 terms for document 780723 are:
gold : 0.2001138697660808
lme : 0.2001138697660808
cash : 0.2001138697660808
quarter : 0.2001138697660808
network : 0.14889311892370607
toronto : 0.13765786215503525
stock : 0.11893090042213532
trade : 0.11893090042213532
million : 0.11893090042213532
tse : 0.11819047514812066
hi : 0.11819047514812066
lo : 0.11819047514812066
dji : 0.118190475